# Test vtl analytics invocation




In [20]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType, LongType, DecimalType
import os
from pyspark.sql.functions import lit, rand,col, rank, desc, first, last
from pyspark.sql.window import Window


In [2]:
local = True

if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("VTLAnalytic")\
        .getOrCreate()
else:
    spark = SparkSession.builder\
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("VTLAnalytic")\
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:py3.9.7-spark3.2.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "8g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-07-02 10:19:46,263 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [36]:
data=[("A", "XX", 1993, 3, 1.0),
    ("A", "XX", 1994, 4, 9.0),
    ("A", "XX", 1995, 7, 5.0),
    ("A", "XX", 1996, 6, 8.0),
    ("A", "YY", 1993, 9, 3.0),
    ("A", "YY", 1994, 5, 4.0),
    ("A", "YY", 1995, 10, 2.0),
    ("A", "YY", 1996, 2, 7.0)]

schema=StructType([StructField("Id_1",StringType(),True),
                   StructField("Id_2",StringType(),True),
                   StructField("Year",IntegerType(),True),
                   StructField("Me_1",IntegerType(),True),
                   StructField("Me_2",DoubleType(),True)])

df=spark.createDataFrame(data, schema)
df.show()

+----+----+----+----+----+
|Id_1|Id_2|Year|Me_1|Me_2|
+----+----+----+----+----+
|   A|  XX|1993|   3| 1.0|
|   A|  XX|1994|   4| 9.0|
|   A|  XX|1995|   7| 5.0|
|   A|  XX|1996|   6| 8.0|
|   A|  YY|1993|   9| 3.0|
|   A|  YY|1994|   5| 4.0|
|   A|  YY|1995|  10| 2.0|
|   A|  YY|1996|   2| 7.0|
+----+----+----+----+----+



In [10]:
df.printSchema()

root
 |-- Id_1: string (nullable = true)
 |-- Id_2: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Me_1: integer (nullable = true)
 |-- Me_2: double (nullable = true)



# 2. VTL advance window function

## 2.1 Rank 

### 2.1.1 Exp1 : window has partition clause and order by clause


```text
res := ds1 [calc rank_col:= rank ( over ( partition by Id_1, Id_2 order by Year) )]
```

**Note: rank function requires rows must be ordered, so we can't apply rank on clause that only have partition**

In [31]:



partition_col_names=["Id_1","Id_2"]
order_col_names=[col("Year").asc()]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

new_col_name=f"rank_col"


df_rank=df.withColumn(new_col_name,rank().over(win_name)) 
df_rank.show()



+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|rank_col|
+----+----+----+----+----+--------+
|   A|  XX|2000|   3| 1.0|       1|
|   A|  XX|2001|   4| 9.0|       2|
|   A|  XX|2002|   7| 5.0|       3|
|   A|  XX|2003|   6| 8.0|       4|
|   A|  YY|2000|   9| 3.0|       1|
|   A|  YY|2001|   5| 4.0|       2|
|   A|  YY|2002|  10| 2.0|       3|
|   A|  YY|2003|   2| 7.0|       4|
+----+----+----+----+----+--------+



### 2.1.2 Exp2 : window has partition clause and order by clause with desc


```text
res := ds1 [calc rank_col:= rank ( over ( partition by Id_1, Id_2 order by Year desc) )]
```

In [32]:
partition_col_names=["Id_1","Id_2"]
order_col_names=[col("Year").desc()]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

new_col_name=f"rank_col"


df_rank=df.withColumn(new_col_name,rank().over(win_name)) 
df_rank.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|rank_col|
+----+----+----+----+----+--------+
|   A|  XX|2003|   6| 8.0|       1|
|   A|  XX|2002|   7| 5.0|       2|
|   A|  XX|2001|   4| 9.0|       3|
|   A|  XX|2000|   3| 1.0|       4|
|   A|  YY|2003|   2| 7.0|       1|
|   A|  YY|2002|  10| 2.0|       2|
|   A|  YY|2001|   5| 4.0|       3|
|   A|  YY|2000|   9| 3.0|       4|
+----+----+----+----+----+--------+



### 2.1.3 Exp3 : window has partition clause, order by clause and data points


```text
res := ds1 [calc rank_col:= rank ( over ( partition by Id_1, Id_2 order by Year desc data points between unbounded preceding and current data point) )]
```

**Note rank function can't not take rolling window such as [-1,1]. It requires [unboundedPreceding,currentRow]**. This window specification has no effect, because it's identical to default rolling window definition. So no need to use `data points` and `range between` in rank

In [13]:
partition_col_names=["Id_1","Id_2"]
order_col_names=[col("Year")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(Window.unboundedPreceding,Window.currentRow)

new_col_name=f"rank_col"


df_rank=df.withColumn(new_col_name,rank().over(win_name)) 
df_rank.show()


+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|rank_col|
+----+----+----+----+----+--------+
|   A|  XX|2000|   3| 1.0|       1|
|   A|  XX|2001|   4| 9.0|       2|
|   A|  XX|2002|   7| 5.0|       3|
|   A|  XX|2003|   6| 8.0|       4|
|   A|  YY|2000|   9| 3.0|       1|
|   A|  YY|2001|   5| 4.0|       2|
|   A|  YY|2002|  10| 2.0|       3|
|   A|  YY|2003|   5| 7.0|       4|
+----+----+----+----+----+--------+



## 2.2 First

### 2.2.1 Exp1 : window has partition clause


```text
res :=  first_value ( ds1 over ( partition by Id_1, Id_2) )
```


In [18]:
from pyspark.sql.functions import first, last

partition_col_names=["Id_1","Id_2"]


win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"first_{target_col_name1}"
new_col_name2=f"first_{target_col_name2}"

df_first=df.withColumn(new_col_name1,first(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,first(target_col_name2).over(win_name))


df_first.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|first_Me_1|first_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         3|       1.0|
|   A|  XX|2001|   4| 9.0|         3|       1.0|
|   A|  XX|2002|   7| 5.0|         3|       1.0|
|   A|  XX|2003|   6| 8.0|         3|       1.0|
|   A|  YY|2000|   9| 3.0|         9|       3.0|
|   A|  YY|2001|   5| 4.0|         9|       3.0|
|   A|  YY|2002|  10| 2.0|         9|       3.0|
|   A|  YY|2003|   5| 7.0|         9|       3.0|
+----+----+----+----+----+----------+----------+



### 2.2.2 Exp2 : window has partition clause and order by clause

vtl query

```text
res :=  first_value ( ds1 over ( partition by Id_1, Id_2 order by Year desc) )
```

In [19]:


partition_col_names=["Id_1","Id_2"]

order_col_names=[col("Year").desc()]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"first_{target_col_name1}"
new_col_name2=f"first_{target_col_name2}"

df_first=df.withColumn(new_col_name1,first(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,first(target_col_name2).over(win_name))


df_first.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|first_Me_1|first_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2003|   6| 8.0|         6|       8.0|
|   A|  XX|2002|   7| 5.0|         6|       8.0|
|   A|  XX|2001|   4| 9.0|         6|       8.0|
|   A|  XX|2000|   3| 1.0|         6|       8.0|
|   A|  YY|2003|   5| 7.0|         5|       7.0|
|   A|  YY|2002|  10| 2.0|         5|       7.0|
|   A|  YY|2001|   5| 4.0|         5|       7.0|
|   A|  YY|2000|   9| 3.0|         5|       7.0|
+----+----+----+----+----+----------+----------+



### 2.2.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := first_value ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [21]:
partition_col_names=["Id_1"]

order_col_names=[col("Id_2")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"first_{target_col_name1}"
new_col_name2=f"first_{target_col_name2}"

df_first=df.withColumn(new_col_name1,first(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,first(target_col_name2).over(win_name))


df_first.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|first_Me_1|first_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         3|       1.0|
|   A|  XX|2001|   4| 9.0|         3|       1.0|
|   A|  XX|2002|   7| 5.0|         3|       1.0|
|   A|  XX|2003|   6| 8.0|         4|       9.0|
|   A|  YY|2000|   9| 3.0|         7|       5.0|
|   A|  YY|2001|   5| 4.0|         6|       8.0|
|   A|  YY|2002|  10| 2.0|         9|       3.0|
|   A|  YY|2003|   5| 7.0|         5|       4.0|
+----+----+----+----+----+----------+----------+



### 2.2.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := first_value ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [22]:
partition_col_names=["Id_1"]

order_col_names=[col("Year")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"first_{target_col_name1}"
new_col_name2=f"first_{target_col_name2}"

df_first=df.withColumn(new_col_name1,first(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,first(target_col_name2).over(win_name))


df_first.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|first_Me_1|first_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         3|       1.0|
|   A|  YY|2000|   9| 3.0|         3|       1.0|
|   A|  XX|2001|   4| 9.0|         3|       1.0|
|   A|  YY|2001|   5| 4.0|         3|       1.0|
|   A|  XX|2002|   7| 5.0|         4|       9.0|
|   A|  YY|2002|  10| 2.0|         4|       9.0|
|   A|  XX|2003|   6| 8.0|         7|       5.0|
|   A|  YY|2003|   5| 7.0|         7|       5.0|
+----+----+----+----+----+----------+----------+



## 2.3 Last

The spark last does not correspond the specification of VTL. So here we just reverse the order and use first() to simulate last() function. For more details, please visit

### 2.3.1 Exp1 : window has partition clause


```text
res :=  last_value ( ds1 over ( partition by Id_1, Id_2) )
```


In [23]:
partition_col_names=["Id_1","Id_2"]


win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"last_{target_col_name1}"
new_col_name2=f"last_{target_col_name2}"

df_last=df.withColumn(new_col_name1,last(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,last(target_col_name2).over(win_name))


df_last.show()

+----+----+----+----+----+---------+---------+
|Id_1|Id_2|Year|Me_1|Me_2|last_Me_1|last_Me_2|
+----+----+----+----+----+---------+---------+
|   A|  XX|2000|   3| 1.0|        6|      8.0|
|   A|  XX|2001|   4| 9.0|        6|      8.0|
|   A|  XX|2002|   7| 5.0|        6|      8.0|
|   A|  XX|2003|   6| 8.0|        6|      8.0|
|   A|  YY|2000|   9| 3.0|        5|      7.0|
|   A|  YY|2001|   5| 4.0|        5|      7.0|
|   A|  YY|2002|  10| 2.0|        5|      7.0|
|   A|  YY|2003|   5| 7.0|        5|      7.0|
+----+----+----+----+----+---------+---------+



### 2.3.2 Exp2 : window has partition clause and order by clause

vtl query

```text
res :=  last_value ( ds1 over ( partition by Id_1, Id_2 order by Year desc) )
```

In [33]:
partition_col_names=["Id_1","Id_2"]
order_col_names=[col("Year").desc()]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"last_{target_col_name1}"
new_col_name2=f"last_{target_col_name2}"

df_last=df.withColumn(new_col_name1,last(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,last(target_col_name2).over(win_name))


df_last.show()

+----+----+----+----+----+---------+---------+
|Id_1|Id_2|Year|Me_1|Me_2|last_Me_1|last_Me_2|
+----+----+----+----+----+---------+---------+
|   A|  XX|2003|   6| 8.0|        6|      8.0|
|   A|  XX|2002|   7| 5.0|        7|      5.0|
|   A|  XX|2001|   4| 9.0|        4|      9.0|
|   A|  XX|2000|   3| 1.0|        3|      1.0|
|   A|  YY|2003|   2| 7.0|        2|      7.0|
|   A|  YY|2002|  10| 2.0|       10|      2.0|
|   A|  YY|2001|   5| 4.0|        5|      4.0|
|   A|  YY|2000|   9| 3.0|        9|      3.0|
+----+----+----+----+----+---------+---------+



### 2.3.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := last_value ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [35]:
partition_col_names=["Id_1"]
order_col_names=[col("Id_2")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"last_{target_col_name1}"
new_col_name2=f"last_{target_col_name2}"

df_last=df.withColumn(new_col_name1,last(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,last(target_col_name2).over(win_name))


df_last.show()

+----+----+----+----+----+---------+---------+
|Id_1|Id_2|Year|Me_1|Me_2|last_Me_1|last_Me_2|
+----+----+----+----+----+---------+---------+
|   A|  XX|2000|   3| 1.0|        7|      5.0|
|   A|  XX|2001|   4| 9.0|        6|      8.0|
|   A|  XX|2002|   7| 5.0|        9|      3.0|
|   A|  XX|2003|   6| 8.0|        5|      4.0|
|   A|  YY|2000|   9| 3.0|       10|      2.0|
|   A|  YY|2001|   5| 4.0|        2|      7.0|
|   A|  YY|2002|  10| 2.0|        2|      7.0|
|   A|  YY|2003|   2| 7.0|        2|      7.0|
+----+----+----+----+----+---------+---------+



### 2.3.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := last_value ( ds1 over ( partition by Id_1, Id_2 order by Year range between -1 and 1) )
```

In [37]:
partition_col_names=["Id_1","Id_2"]
order_col_names=[col("Year")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"last_{target_col_name1}"
new_col_name2=f"last_{target_col_name2}"

df_last=df.withColumn(new_col_name1,last(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,last(target_col_name2).over(win_name))


df_last.show()

+----+----+----+----+----+---------+---------+
|Id_1|Id_2|Year|Me_1|Me_2|last_Me_1|last_Me_2|
+----+----+----+----+----+---------+---------+
|   A|  XX|1993|   3| 1.0|        4|      9.0|
|   A|  XX|1994|   4| 9.0|        7|      5.0|
|   A|  XX|1995|   7| 5.0|        6|      8.0|
|   A|  XX|1996|   6| 8.0|        6|      8.0|
|   A|  YY|1993|   9| 3.0|        5|      4.0|
|   A|  YY|1994|   5| 4.0|       10|      2.0|
|   A|  YY|1995|  10| 2.0|        2|      7.0|
|   A|  YY|1996|   2| 7.0|        2|      7.0|
+----+----+----+----+----+---------+---------+



#### 2.3.5 Exp to compare with official example


```text
res :=  = last_value ( ds1 over ( partition by Id_1, Id_2 order by Id_3 data points between 1 preceding and 1 following ) 
```

page 178, the official doc is wrong

results in: DS_r

```text
Id_1 Id_2 Id_3 Me_1 Me_2
A XX 1993 4 9
A XX 1994 7 9
A XX 1995 7 9
A XX 1996 7 8
A YY 1993 9 4
A YY 1994 10 4
A YY 1995 10 7
A YY 1996 10 7
```

In [38]:
data1=[("A", "XX", 1993, 3, 1.0),
    ("A", "XX", 1994, 4, 9.0),
    ("A", "XX", 1995, 7, 5.0),
    ("A", "XX", 1996, 6, 8.0),
    ("A", "YY", 1993, 9, 3.0),
    ("A", "YY", 1994, 5, 4.0),
    ("A", "YY", 1995, 10, 2.0),
    ("A", "YY", 1996, 2, 7.0)]

schema1=StructType([StructField("Id_1",StringType(),True),
                   StructField("Id_2",StringType(),True),
                   StructField("Year",IntegerType(),True),
                   StructField("Me_1",IntegerType(),True),
                   StructField("Me_2",DoubleType(),True)])

df1=spark.createDataFrame(data1, schema1)
df1.show()

+----+----+----+----+----+
|Id_1|Id_2|Year|Me_1|Me_2|
+----+----+----+----+----+
|   A|  XX|1993|   3| 1.0|
|   A|  XX|1994|   4| 9.0|
|   A|  XX|1995|   7| 5.0|
|   A|  XX|1996|   6| 8.0|
|   A|  YY|1993|   9| 3.0|
|   A|  YY|1994|   5| 4.0|
|   A|  YY|1995|  10| 2.0|
|   A|  YY|1996|   2| 7.0|
+----+----+----+----+----+



In [39]:
partition_col_names=["Id_1","Id_2"]

order_col_names=[col("Year")]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"last_{target_col_name1}"
new_col_name2=f"last_{target_col_name2}"

df_last1=df1.withColumn(new_col_name1,last(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,last(target_col_name2).over(win_name))


df_last1.show()

+----+----+----+----+----+---------+---------+
|Id_1|Id_2|Year|Me_1|Me_2|last_Me_1|last_Me_2|
+----+----+----+----+----+---------+---------+
|   A|  XX|1993|   3| 1.0|        4|      9.0|
|   A|  XX|1994|   4| 9.0|        7|      5.0|
|   A|  XX|1995|   7| 5.0|        6|      8.0|
|   A|  XX|1996|   6| 8.0|        6|      8.0|
|   A|  YY|1993|   9| 3.0|        5|      4.0|
|   A|  YY|1994|   5| 4.0|       10|      2.0|
|   A|  YY|1995|  10| 2.0|        2|      7.0|
|   A|  YY|1996|   2| 7.0|        2|      7.0|
+----+----+----+----+----+---------+---------+



## 1.4 Lead()

DS_r := lead ( DS_1 , 1 over ( partition by Id_1 , Id_2 order by Id_3 ) )

Input: DS_1
```text
Id_1 Id_2 Id_3 Me_1 Me_2
A XX 1993 3 1
A XX 1994 4 9
A XX 1995 7 5
A XX 1996 6 8
A YY 1993 9 3
A YY 1994 5 4
A YY 1995 10 2
A YY 1996 2 7
```

Output : DS_r
```text
Id_1 Id_2 Id_3 Me_1 Me_2
A XX 1993 4 9
A XX 1994 7 5
A XX 1995 6 8
A XX 1996 NULL NULL
A YY 1993 5 4
A YY 1994 10 2
A YY 1995 2 7
A YY 1996 NULL NULL
```


In [27]:
from pyspark.sql.functions import lead, lag
partition_col_name=["Id_1","Id_2"]
win_name=Window.partitionBy(partition_col_name)

order_col_name=[col("Id_3").asc()]
win_name_order=win_name.orderBy(order_col_name)
df_input=df
col_names=["Me_1","Me_2"]
step=1
for col_name in col_names:
    lead_col_name=f"lead_{col_name}"
    df_input=df_input.select("*",lead(col_name,1).over(win_name_order).alias(lead_col_name)).drop(col_name).withColumnRenamed(lead_col_name,col_name)
df_input.show()

+----+----+----+----+----+
|Id_1|Id_2|Id_3|Me_1|Me_2|
+----+----+----+----+----+
|   A|  YY|2000|   5|   4|
|   A|  YY|2001|  10|   2|
|   A|  YY|2002|   5|   7|
|   A|  YY|2003|null|null|
|   A|  XX|2000|   4|   9|
|   A|  XX|2001|   7|   5|
|   A|  XX|2002|   6|   8|
|   A|  XX|2003|null|null|
+----+----+----+----+----+



## 1.5 Lag()

In the ordered set of Data Points of the current partition, the operator returns the value(s) taken from the Data Point at the specified physical offset prior to the current Data Point.

If defaultValue is not specified then the value returned when the offset goes outside the partition is NULL.

VTL query: DS_r := lag ( DS_1 , 1 over ( partition by Id_1 , Id_2 order by Id_3 ) ) results in:
Input: DS_1
```text
Id_1 Id_2 Id_3 Me_1 Me_2
A XX 1993 3 1
A XX 1994 4 9
A XX 1995 7 5
A XX 1996 6 8
A YY 1993 9 3
A YY 1994 5 4
A YY 1995 10 2
A YY 1996 2 7

```

Output: DS_r
```text
Id_1 Id_2 Id_3 Me_1 Me_2
A XX 1993 NULL NULL
A XX 1994 3 1
A XX 1995 4 9
A XX 1996 7 5
A YY 1993 NULL NULL
A YY 1994 9 3
A YY 1995 5 4
A YY 1996 10 2
```


In [28]:
from pyspark.sql.functions import lead, lag
partition_col_name=["Id_1","Id_2"]
win_name=Window.partitionBy(partition_col_name)

order_col_name=[col("Id_3").asc()]
win_name_order=win_name.orderBy(order_col_name)
df_input=df
col_names=["Me_1","Me_2"]
step=1
for col_name in col_names:
    lead_col_name=f"lead_{col_name}"
    df_input=df_input.select("*",lag(col_name,1).over(win_name_order).alias(lead_col_name)).drop(col_name).withColumnRenamed(lead_col_name,col_name)
df_input.show()

+----+----+----+----+----+
|Id_1|Id_2|Id_3|Me_1|Me_2|
+----+----+----+----+----+
|   A|  YY|2000|null|null|
|   A|  YY|2001|   9|   3|
|   A|  YY|2002|   5|   4|
|   A|  YY|2003|  10|   2|
|   A|  XX|2000|null|null|
|   A|  XX|2001|   3|   1|
|   A|  XX|2002|   4|   9|
|   A|  XX|2003|   7|   5|
+----+----+----+----+----+



## 1.6 ratio_to_report()

The operator returns the ratio between the value of the current Data Point and the sum of the values of the partition which the current Data Point belongs to.

ETL example

DS_r := ratio_to_report ( DS_1 over ( partition by Id_1, Id_2 ) )

Note, here even though the above vtl request does not specify which column we need to apply the ratio_to_report on. The result calculate the ratio_to_report on column, Me_1 and Me_2. Because VTL by default will apply all function without parameter on all column which has property **Measurement**

Input DS_1

```text

Id_1 Id_2 Id_3 Me_1 Me_2
A XX 2000 3 1
A XX 2001 4 3
A XX 2002 7 5
A XX 2003 6 1
A YY 2000 12 0
A YY 2001 8 8
A YY 2002 6 5
A YY 2003 14 -3

```

output: DS_r

```text
Id_1 Id_2 Id_3 Me_1 Me_2
A YY 2000 0.3 0
A YY 2001 0.2 0.8
A YY 2002 0.15 0.5
A YY 2003 0.35 -0.3
A XX 2000 0.15 0,1
A XX 2001 0.2 0.3
A XX 2002 0.35 0.5
A XX 2003 0.3 0.1

```

In [9]:
data1=[
    ("A", "XX", 2001, 4, 3),
    ("A", "XX", 2002, 7, 5),
    ("A", "XX", 2000, 3, 1),
    ("A", "XX", 2003, 6, 1),
    ("A", "YY", 2000, 12, 0),
    ("A", "YY", 2001, 8, 8),
    ("A", "YY", 2002, 6, 5),
    ("A", "YY", 2003, 14, -3)]

schema1=StructType([StructField("Id_1",StringType(),True),
                   StructField("Id_2",StringType(),True),
                   StructField("Id_3",IntegerType(),True),
                   StructField("Me_1",IntegerType(),True),
                   StructField("Me_2",IntegerType(),True)])

df1=spark.createDataFrame(data1, schema1)
df1.show()

+----+----+----+----+----+
|Id_1|Id_2|Id_3|Me_1|Me_2|
+----+----+----+----+----+
|   A|  XX|2000|   3|   1|
|   A|  XX|2001|   4|   3|
|   A|  XX|2002|   7|   5|
|   A|  XX|2003|   6|   1|
|   A|  YY|2000|  12|   0|
|   A|  YY|2001|   8|   8|
|   A|  YY|2002|   6|   5|
|   A|  YY|2003|  14|  -3|
+----+----+----+----+----+



In [19]:
from pyspark.sql.functions import sum
partition_col_name=["Id_1","Id_2"]
win_name=Window.partitionBy(partition_col_name)

order_col_name=[col("Id_3").asc()]
win_name_order=win_name.orderBy(rand(100))

col_names=["Me_1","Me_2"]
for col_name in col_names:
    total_col_name=f"total_{col_name}"
    df1=df1.withColumn(total_col_name,sum(col_name).over(win_name)).withColumn(f"ratio_{col_name}",col(col_name)/col(f"total_{col_name}")).drop(total_col_name).drop(col_name).withColumnRenamed(total_col_name,col_name)
df1.show()

+----+----+----+----------+----------+
|Id_1|Id_2|Id_3|ratio_Me_1|ratio_Me_2|
+----+----+----+----------+----------+
|   A|  YY|2000|       0.3|       0.0|
|   A|  YY|2001|       0.2|       0.8|
|   A|  YY|2002|      0.15|       0.5|
|   A|  YY|2003|      0.35|      -0.3|
|   A|  XX|2000|      0.15|       0.1|
|   A|  XX|2001|       0.2|       0.3|
|   A|  XX|2002|      0.35|       0.5|
|   A|  XX|2003|       0.3|       0.1|
+----+----+----+----------+----------+

